<a href="https://colab.research.google.com/github/Yonch1/Machine-Learning-2/blob/main/Yonatan_Shirman_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***INTRO***

this is the code representing the best results I managed to achieve with the given layers (my ID number).

All of my insights from each experiment are written in the attached file in my github:

this notebook with the code can be found as weel in the same git.

In [40]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Load the DB

In [41]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples


print(num_of_classes)
print(image_shape)
print(train_size)
print(test_size)



10
(28, 28, 1)
60000
10000


Data preprocessing

In [42]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

 Deeper network relu + regularization l2

In [43]:
layers = [
  tf.keras.layers.Flatten(input_shape=image_shape),

  tf.keras.layers.Dense(46, kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
  tf.keras.layers.Activation('relu'),

  tf.keras.layers.Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
  tf.keras.layers.Activation('relu'),

  tf.keras.layers.Dense(87, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.Activation('relu'),

  tf.keras.layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.1)),
  tf.keras.layers.Activation('relu'),

  tf.keras.layers.Dense(num_of_classes),
  tf.keras.layers.Softmax()
  ]

Define sequential model

In [44]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 46)                  │          36,110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_25 (Activation)           │ (None, 46)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 2)                   │              94 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_26 (Activation)           │ (None, 2)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 87)                  │             261 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_27 (Activation)           │ (None, 87)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 8)                   │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_28 (Activation)           │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 10)                  │              90 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_6 (Softmax)                  │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 37,259 (145.54 KB)

 Trainable params: 37,259 (145.54 KB)

 Non-trainable params: 0 (0.00 B)

Train the model

In [45]:
model.fit(ds_train,
          epochs=50,
          validation_data=ds_test)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 1.8461 - sparse_categorical_accuracy: 0.4530 - val_loss: 0.9252 - val_sparse_categorical_accuracy: 0.7626
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8672 - sparse_categorical_accuracy: 0.7928 - val_loss: 0.7591 - val_sparse_categorical_accuracy: 0.8311
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7131 - sparse_categorical_accuracy: 0.8460 - val_loss: 0.6400 - val_sparse_categorical_accuracy: 0.8734
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.6326 - sparse_categorical_accuracy: 0.8717 - val_loss: 0.5845 - val_sparse_categorical_accuracy: 0.8851
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5677 - sparse_categorical_accuracy: 0.8896 - val_loss: 0.5525 - val_sparse_categorical_accuracy: 0.8967
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5213 - sparse_categorical_accuracy: 0.9013 - val_loss: 0.5279 - val_sparse_categorical_accuracy

KeyboardInterrupt: 